<a href="https://colab.research.google.com/github/alex-nuclearboy/goit-python-ds-project/blob/main/toxic_albert_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AlbertTokenizer, AlbertForSequenceClassification, Trainer, TrainingArguments
import torch
import numpy as np

In [ ]:
# Завантаження датасету з Google Диску
from google.colab import drive
drive.mount('/content/drive')

# Шлях до очищеного датасету
dataset_path = '/content/drive/My Drive/toxic_comments_data/train_claud_2_0.csv'

# Завантаження датасету у DataFrame
df = pd.read_csv(dataset_path)

Mounted at /content/drive


In [ ]:
# Перевірка наявності стовпців
print(df.columns)

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate', 'token_count'],
      dtype='object')


In [ ]:
# Обираємо потрібні стовпці
df['label'] = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values.tolist()

In [ ]:
# Перетворення міток у формат float32
df['label'] = df['label'].apply(lambda x: np.array(x, dtype=np.float32))

In [ ]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,token_count,label
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0,54,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,000103f0d9cfb60f,daww he matches this background colour im seem...,0,0,0,0,0,0,20,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,000113f07ec002fd,"hey man, im really not trying to edit war. its...",0,0,0,0,0,0,48,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
3,0001b41b1c6bb37e,more i cant make any real suggestions on impro...,0,0,0,0,0,0,94,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
4,0001d958c54c6e35,"you, sir, are my hero. any chance you remember...",0,0,0,0,0,0,17,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"


In [ ]:
# Конвертація у формат Dataset
dataset = Dataset.from_pandas(df[['comment_text', 'label']])

In [ ]:
# Розподіл на тренувальний і тестовий набори
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [ ]:
# Ініціалізуємо токенізатор
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Функція для токенізації даних
def tokenize_function(examples):
    return tokenizer(examples['comment_text'], padding='max_length', truncation=True)

In [ ]:
# Токенізуємо тренувальний та тестовий набори
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/126982 [00:00<?, ? examples/s]

Map:   0%|          | 0/31746 [00:00<?, ? examples/s]

In [ ]:
# Функція для токенізації даних
def tokenize_function(examples):
    return tokenizer(examples['comment_text'], padding='max_length', truncation=True)

In [ ]:
# Токенізуємо тренувальний та тестовий набори
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/126396 [00:00<?, ? examples/s]

Map:   0%|          | 0/31600 [00:00<?, ? examples/s]

In [ ]:
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
# Завантаження моделі
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=6, problem_type="multi_label_classification")

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Налаштування Data Collator для багатокласових міток
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# Налаштування параметрів тренування
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # gradient_accumulation_steps=4,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True  # Увімкнення half-precision
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Створення тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
# Перевірка типу міток
print(train_dataset[0]['label'], train_dataset[0]['label'].dtype)

tensor([0., 0., 0., 0., 0., 0.]) torch.float32


In [ ]:
print(df[['comment_text', 'label']].head())

                                        comment_text  \
0  explanation why the edits made under my userna...   
1  daww he matches this background colour im seem...   
2  hey man, im really not trying to edit war. its...   
3  more i cant make any real suggestions on impro...   
4  you, sir, are my hero. any chance you remember...   

                            label  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  


In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
# Запуск тренування
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.048500,0.058882
2,0.050600,0.051209
3,0.038700,0.048810


TrainOutput(global_step=47619, training_loss=0.05118279625420739, metrics={'train_runtime': 14783.6619, 'train_samples_per_second': 25.768, 'train_steps_per_second': 3.221, 'total_flos': 9107486011625472.0, 'train_loss': 0.05118279625420739, 'epoch': 3.0})

In [ ]:
output_dir = '/content/drive/MyDrive/model_output_2_0/'

# Перевіряємо чи існує директорія, якщо ні, створюємо
import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Збереження моделі
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('/content/drive/MyDrive/model_output_2_0/tokenizer_config.json',
 '/content/drive/MyDrive/model_output_2_0/special_tokens_map.json',
 '/content/drive/MyDrive/model_output_2_0/spiece.model',
 '/content/drive/MyDrive/model_output_2_0/added_tokens.json')

In [ ]:
# Оцінка моделі на тестовому наборі
results = trainer.evaluate()

# Виведення результатів
print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 0.04880952090024948, 'eval_runtime': 429.2357, 'eval_samples_per_second': 73.959, 'eval_steps_per_second': 9.247, 'epoch': 3.0}


In [ ]:
# Виведення параметрів моделі
for name, param in model.named_parameters():
    print(f"Parameter name: {name}, size: {param.size()}")

Parameter name: albert.embeddings.word_embeddings.weight, size: torch.Size([30000, 128])
Parameter name: albert.embeddings.position_embeddings.weight, size: torch.Size([512, 128])
Parameter name: albert.embeddings.token_type_embeddings.weight, size: torch.Size([2, 128])
Parameter name: albert.embeddings.LayerNorm.weight, size: torch.Size([128])
Parameter name: albert.embeddings.LayerNorm.bias, size: torch.Size([128])
Parameter name: albert.encoder.embedding_hidden_mapping_in.weight, size: torch.Size([768, 128])
Parameter name: albert.encoder.embedding_hidden_mapping_in.bias, size: torch.Size([768])
Parameter name: albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.weight, size: torch.Size([768])
Parameter name: albert.encoder.albert_layer_groups.0.albert_layers.0.full_layer_layer_norm.bias, size: torch.Size([768])
Parameter name: albert.encoder.albert_layer_groups.0.albert_layers.0.attention.query.weight, size: torch.Size([768, 768])
Parameter name: albert.encod

In [ ]:
# Збереження тренованої моделі через Trainer
trainer.save_model(output_dir)